In [ ]:
  """
  17, 18
 graph can be represented in the form of set of vertices and edges:  G = (V, E)
 
 v1 --> v2: directed graph or digraph
 v1 --- v2: non-directed graph
 some grpahs are associated with weights or cost for the edges called weighted graph.
 
 An edge with single vertex called self loop.
 multi-edge graph
 
 if there is no self loops and multi edges then we call the graph as simple graph.
 ****************************************************************************************
 
 properties of graphs:
 1. Number of edges: if |V| = n, then 0 <= |E| <= n(n-1) --> if the graph is directed.
 2. Number of edges:                  0 <= |E| <= n(n-1)//2 --> undirected graph

 dense graph:   too many edges
 sparse graph:  too few edges
 *****************************************************************************************
 Path: sequence of vertices where each adjacent pair is connected by an edge.
 Simple Path: a path in which no vertices(and thus no edges) are repeated.
 
 strongly connected graph: if there is path exist from any vertex to any other vertex in the graph.
 
 closed walk: starts and end at same vertex
 cycle: no repetetion other than start and end
 acyclic: a graph with no cycle.
 
 Directed Acyclic Graph (DAG): edges are directed and there is no cycle.
 
 Graph representation: adjacency matrix and adjacency list
                       incident matrix and incident list
 
 Adjacency matrix: removing an edge --> O(1) T
                   u and v connected --> O(1)T
                   consumes more space O(v^2)S
                   adding vertex to the graph also takes O(v^2)T -> adding new vertex
                   adding an edge: O(1)T
                   removing vertex: O((|V|+1)^2) --> O(|V|^2)
                   querying: O(1)
                   
 Adjancency list: --> an array of lists are used. arr[i] represent the list of adjacent nodes of ith vertex 
                  --> space complexity --> O(|V|+|E|)
                  --> adding an edge and vertex can be done with O(1)T
                  --> removing vertex --> O(|E|+|V|)
                  --> removing edge --> O(|E|)
                  --> querying: O(|V|)
                  
  
 """

In [3]:
# the keys to the dictionary are Node's of the graph. values are the adjacent node's that cane be connected by edge.

def generate_edges(graph):
    edges = []
    for node in graph:
        for node1 in graph[node]:
            edges.append((node, node1))
    return edges

def find_isolated_nodes(graph):
    edges = []
    for node in graph:
        if len(graph[node]) == 0:
            edges.append(node)
    return edges

graph = { "a" : ["c"],
          "b" : ["c", "e"],
          "c" : ["a", "b", "d", "e"],
          "d" : ["c"],
          "e" : ["c", "b"],
          "f" : []
        }
print(generate_edges(graph))
print(find_isolated_nodes(graph))

[('a', 'c'), ('b', 'c'), ('b', 'e'), ('c', 'a'), ('c', 'b'), ('c', 'd'), ('c', 'e'), ('d', 'c'), ('e', 'c'), ('e', 'b')]
['f']


In [16]:
class Graph(object):
    def __init__(self, graph_dict = None):
        if graph_dict is None:
            graph_dict = {}
        self.graph_dict = graph_dict
        
    def vertices(self):
        return list(self.graph_dict.keys())
    
    def generate_edges(self):
        edges = []
        for node in self.graph_dict:
            for node1 in self.graph_dict[node]:
                edges.append((node, node1))
        return edges

    def find_isolated_nodes(self):
        edges = []
        for node in self.graph_dict:
            if len(self.graph_dict[node]) == 0:
                edges.append(node)
        return edges
    
    def edges(self):
        return self.generate_edges()
    
    def add_vertex(self, vertex):
        # if the vertex "vertex" is not in self.graph_dict a key "vertex" with an empty list as a value added to the dictionary.
        # otherwise nothing has to done.
        
        if vertex not in self.graph_dict:
            self.graph_dict[vertex] = []
            
    def add_edge(self, edge):
        edge = set(edge)
        (vertex1, vertex2) = tuple(edge)
        if vertex1 in self.graph_dict:
            self.graph_dict[vertex1].append(vertex2)
        else:
            self.graph_dict[vertex1] = [vertex2]
            
    def find_path(self, start, end, path = None):
        if path == None: path = []
        path = path + [start]
        if start == end: return path
        if start not in self.graph_dict:
            return None
        p = []
        for vertex in self.graph_dict[start]:
            if vertex not in path:
                extended_path = self.find_path(vertex, end, path)
                
                if extended_path: return extended_path
        return None
    
    def find_all_paths(self, start, end, path = []):
        path = path + [start]
        if start == end: return [path]
        if start not in self.graph_dict: return []
        paths = []
        for vertex in self.graph_dict[start]:
            if vertex not in path:
                extended_path = self.find_all_paths(vertex, end, path)
                for p in extended_path: paths.append(p)
        return paths
    
    # degree of the graph:
    """ Degree of vertex is no of edges connecting to it. if there is any self loop we'll count it as twice.
        if all degrees are same we call that graph as regular graph.--> sum of all the degrees = 2*number of edges"""
    def vertex_degree(self, vertex):
        if vertex not in self.graph_dict: return None
        else:
            degree = 0
            for node in self.graph_dict[vertex]:
                if node == vertex: degree += 2
                else: degree += 1
        return degree
    
    def isolated_vertices(self):
        res = []
        for vertex in self.graph_dict:
            if not self.graph_dict[vertex]:
                res.append(vertex)
        return vertex
                
    def delta(self): # finding the minimum degree of a graph
        res = float("inf")
        for vertex in self.graph_dict:
            v_d = self.vertex_degree(vertex)
            if v_d < res:
                res = v_d
        return res
    def Delta(self): #finding the maximum degree of a graph
        res = 0
        for vertex in self.graph_dict:
            v_d = self.vertex_degree(vertex)
            if v_d > res: res = v_d
        return res
    
    # Degree sequence: Degree sequence of undirected graph -> sequence of it's vertex degrees in non-increasing order
    def degree_sequence(self):
        res = []
        for vertex in self.graph_dict:
            res.append(self.vertex_degree(vertex))
        res.sort(reverse = True)
        return res
    
    """ Erdos-Gallai Theorem: it states that non-increasing sequence of n numbers is the degree sequence of simple graph if and
        only if sum of the sequence is even and satisfy the given condition."""
    def erdos_Gallai(self, seq):
        if sum(seq)%2 != 0: return False
        if self.is_degree_sequence(seq):
            for k in range(1, len(seq)+1):
                left = sum(seq[:k])
                right = k*(k-1) + sum([min(x, k) for x in seq[k:]])
                if left > right: return False
        else:
            return False
        return True
    def is_degree_sequence(self, seq):
        for i in range(1, len(seq)):
            if seq[i-1]<seq[i]: return False
        return True
    
    # Graph Density: it is the ratio of the number of edges of the given graph to the total number of the edges. if density of 
    # graph is 1 then it means complete graph. 0 --> isolated graph.
    # maximum number of edges: V*(V-1)/2
    # for undirected simple graphs --> D = 2*E/(V(V-1))
    
    def density(self):
        V = len(self.graph_dict.keys())
        E = len(self.edges())
        return 2.0 * E / (V *(V - 1))
    
    # connected graph: every pair in the graph is connected then we call it as connected graph.
    def is_connected(self, vertices_encountered = None, start_vertex = None):
        if vertices_encountered == None: vertices_encountered = set()
        vertices = list(self.graph_dict.keys())
        if not start_vertex: start_vertex = vertices[0]
        vertices_encountered.add(start_vertex)
        if len(vertices_encountered)!=len(vertices):
            for vertex in self.graph_dict[start_vertex]:
                if vertex not in vertices_encountered:
                    if self.is_connected(vertices_encountered, vertex): return True        
        else:
            return True
        return False   
    # distance: the distance dist between two vertices of the graph is the length of shortest path between these two vertices.
    # dist("a", "f") = 3
    # Eccentricity of vertex s of graph g is maximal distance to every other vertex of the graph.e(s) = max({dist(s,v)|v belongs to V})
    
    # Diameter of a graph: maximum eccentricity of any vertex in the graph. simply diameter is the length of the shortest path
    # between most distanced nodes.
    # Radius: minimum of the all the eccentricities of the graph.
    
    def diameter_graph(self):
        v = self.vertices()
        pairs = [(v[i], v[j]) for i in range(len(v)-1) for j in range(1, len(v))]
        smallest_paths = []
        for (s,e) in pairs:
            paths = self.find_all_paths(s, e)
            smallest = sorted(paths, key=len)[0]
            smallest_paths.append(smallest)
        smallest_paths.sort(key=len)
        return len(smallest_paths[-1])-1
        
g = { "a" : ["c"],
      "b" : ["c", "e", "f"],
      "c" : ["a", "b", "d", "e"],
      "d" : ["c"],
      "e" : ["c", "b", "f"],
      "f" : ["b", "e"]
        }
graph = Graph(g)
#print(graph.vertices())
#print(graph.edges())
#graph.add_vertex("z")
#print(graph.vertices())
#graph.add_edge({"a", "z"})
#print(graph.edges())
#graph.add_edge({"x", "y"})
#print(graph.edges())
print()
#print(graph.find_path("a", "f"))
print(graph.find_all_paths("a", "f"))
#print(graph.vertex_degree("c"))
#print(graph.isolated_vertices())
#print(graph.degree_sequence())
#print(graph.erdos_Gallai([6, 3, 3, 2, 1, 1, 2, 1]))
#print(graph.density())
#print(graph.is_connected())
print(graph.diameter_graph())


[['a', 'c', 'b', 'e', 'f'], ['a', 'c', 'b', 'f'], ['a', 'c', 'e', 'b', 'f'], ['a', 'c', 'e', 'f']]
3


In [ ]:
"""
classes of graph:
1. regular graph: graph with every vertex has of same degree. eg: 1-regular graph, 2-regular graph, k-regular graph.
2. complete graph: every vertex in the graph is connected to every other vertex in the graph.
3. connected graph: if there is a path exist from one vertex to every other vertex in the graph. if it is directed graph then we
                    call that graph as strongly connected graph.
4. planar graph: no two edges will intersect to each other.
5. Bipartite graph: every vertex in one set will connect to every vertex in the other set.
6. Tree: it's also connected graph but no cycle present in the graph.


Mother vertex: vertex which has a path to reach every other vertex in the graph.
"""

In [27]:

def check_arr_is_subarr(arr1, arr2):
    i, j = 0, 0
    while i< len(arr1) and j<len(arr2):
        if arr1[i] == arr1[j]:
            if arr1[i:i+len(arr2)] == arr2:
                return True
            i+=1
            j+=1
        else:
            i+=1
    return False
a = [1,2,3]
b = [2,3]
check_arr_is_subarr(a, b)
            

True

In [1]:
# Breadth First Traversal of Graph: Queue
"""
--> enque starting vertex.
--> while Queue is not empty:
        p = dequeue()
        enque all adjacent elements
        print(p)
        check visited array.
"""
from collections import defaultdict
class Graph:
    def __init__(self):
        self.graph = defaultdict(list)
    
    def addEdge(self, u, v):
        self.graph[u].append(v)
    
    def print_graph(self):
        return self.graph
        
    def BFS(self, start):
        visited = set()
        queue = []
        queue.append(start)
        visited.add(start)
        
        while queue:
            p = queue.pop(0)
            print(p, end = " ")
            
            for i in self.graph[p]:
                if i not in visited:
                    queue.append(i)
                    visited.add(i)
    def BFS_recursive(self, start):
        visited = set()
        queue = [start]
        def helper(queue, visited):
            if not queue: return 
            v = queue.pop(0)
            print(v, end = " ")
            
            for u in self.graph[v]:
                if u not in visited:
                    visited.add(u)
                    queue.append(u)
            if queue: helper(queue, visited)
        helper(queue, visited)
     
    """
    --> push starting vertex
    --> while stack not empty:
            p = top()
            push only one adjacent vertex of p
            print(p)
            check visited array
            if there is no valid vertices for p then pop()
    """
    def DFS_recursive(self, start):
        visited = set()
        def helper(start):
            if start not in visited:
                print(start, end = " ")
                visited.add(start)
                for i in self.graph[start]:
                    helper(i)
        helper(start)
    
    def DFS_iterative(self, start):
        stack, visited = [start], set()
        while stack:
            p = stack.pop()
            if p in visited: continue
            
            visited.add(p)
            print(p, end = " ")
            
            adj = self.graph[p]
            for u in reversed(adj):
                if u not in visited: stack.append(u)
        
g = Graph()
g.addEdge(1,2)
g.addEdge(1, 3)
g.addEdge(1, 4)
g.addEdge(2, 5)
g.addEdge(2, 6)
g.addEdge(4, 7)
g.addEdge(4, 8)
g.addEdge(5, 9)
g.addEdge(5, 10)
g.addEdge(7, 11)
g.addEdge(7, 12)

#print(g.print_graph())
#print(g.BFS(1))
print(g.BFS_recursive(1))
#print(g.DFS_iterative(1))
#print(g.DFS_recursive(1))
        
        


1 2 3 4 5 6 7 8 9 10 11 12 None


In [ ]:
# Depth First Search: Stack
"""
--> push starting vertex
--> while stack not empty:
       p = top()
       push only one adjacent vertex of p
       print(p)
       check visited array
       if there is no valid vertices for p then pop()
"""

In [1]:
def addition(s, a):
    s= list(s)
    for i in range(len(s)):
        if (i % 2) != 0:
            x = str(int(s[i])+a)
            s[i] = x[-1]
        
    return "".join(s)

s = "2235"
a= 1
addition(s, a)

'2336'

In [2]:
# frog crossing the river on leaves:
def function(x, arr):
    # check whether there are leaves in between 0 and x+1 in all the positions
    # simply the minimum time require to cover all the positions with leafs in between 0 and x+1
    res = -1
    res_arr = [0]*(len(arr)+1)
    for i in range(len(arr)):
        res += 1
        res_arr[arr[i]] = 1
        if len(set(res_arr[1:x+1])) == 1:
            return res
    return -1
    
    
    
arr = [2,2,2,2,2]
x = 2
function(x, arr)
    

-1

In [10]:
# return True or False if they are equal if we swap the array elements 

def function(arr1, arr2):
    m = max(arr1)
    n = len(arr1)
    s1, s2 = sum(arr1), sum(arr2)
    d = s2-s1
    print(d)
    if d%2 == 1: return False
    d //= 2
    count = count_elements(arr1, m)
    print(count)
    for i in range(n):
        print("hi")
        if 0 <= arr2[i] - d <= m and count[arr2[i] - d] > 0:
            return True
    return False
    
def count_elements(arr, m):
    count = [0]*(m+1)
    for i in range(len(arr)):
        count[arr[i]]+=1
    return count

arr1 = [5,7,4,6]
arr2 = [1,2,3,8]
function(arr1, arr2)

-8
[0, 0, 0, 0, 1, 1, 1, 1]
hi


True

In [11]:
# prefix sums and suffix sums calculations:

def prefix_sum(arr):
    for i in range(1,len(arr)):
        arr[i] = arr[i]+arr[i-1]
    return arr
arr = [1,2,3,4,5,6]
prefix_sum(arr)

[1, 3, 6, 10, 15, 21]

In [7]:
def function(s, p, q):
    count = []
    for i in range(3):
        count.append([0]*(len(s)+1))
    for index, i in enumerate(s):
        count[0][index+1] = count[0][index] + (i=='A')
        count[1][index+1] = count[1][index] + (i=='C')
        count[2][index+1] = count[2][index] + (i=='G')
    print(count)
    res = []
    for i in range(len(p)):
        start = p[i]
        end = q[i]+1
        if count[0][end]-count[0][start]: res.append(1)
        elif count[1][end]-count[1][start]: res.append(2)
        elif count[2][end]-count[2][start]: res.append(3)
        else: res.append(4)
    return res
s = "CAGCCTA"
p = [2,5,0]
q = [4,5,6]
function(s, p, q)

[[0, 0, 1, 1, 1, 1, 1, 2], [0, 1, 1, 1, 2, 3, 3, 3], [0, 0, 0, 1, 1, 1, 1, 1]]


[2, 4, 1]

In [1]:
# Dijkstra's algorithm: single source shoretest path algorithm
# 1. undirected graph
# 2. directed graph.


True

In [4]:
# count the number of ways to traverse the 2D array:
def count_ways(n, m):
    def compute(x, y):
        if x == y == 0: return 1
        top = 0 if x == 0 else compute(x-1, y)
        left = 0 if y == 0 else compute(x, y-1)
        return top+left
    return compute(n, m)

count_ways(4,4)

""" time complexity is O(nm) and space complexity O(nm)
    --> (n+m-2)!/(n-1)!*(m-1)!
"""


70

In [ ]:
class Graph(object):
    def __init__(self, graph_dict = None):
        if graph_dict is None:
            graph_dict = {}
        self.graph_dict = graph_dict
        
        
g = { 0 : [1, 2],
      1 : [],
      2 : [3, 4],
      3 : [1, 5],
      4 : [5],
      5 : [],
      6 : [7],
      7 : []
        }
graph = Graph(g)

In [9]:
""" if we sort the node data with respect to departure times in descending order, this implies toplogical sorting of the data.
    Means left to right."""

from collections import defaultdict
class Graph:
    def __init__(self):
        self.graph = defaultdict(list)
    
    def addEdge(self, u, v):
        self.graph[u].append(v)
    
    def print_graph(self):
        return self.graph
    
    def count_vertices(self):
        count = 0
        visit = set()
        for i in self.graph:
            if i not in visit:
                visit.add(i)
                count += 1
                for j in self.graph[i]:
                    if j not in visit:
                        visit.add(j)
                        count += 1
        return count        
    
    def arrival_deaparture_times(self):
        N = self.count_vertices()
        print(N)
        def arrival_dep_times(N):
            self.arrival = [None]*N
            self.departure = [None]*N
            self.discovered = [False]*N
            self.time = -1
            def dfs(start):
                self.time += 1
                self.arrival[start] = self.time
                self.discovered[start] = True
                for i in self.graph[start]:
                    if not self.discovered[i]:
                        self.time = dfs(i)
                self.time += 1
                self.departure[start] = self.time
                return self.time 
            for i in range(N):
                if not self.discovered[i]:
                    dfs(i)
            res = {}
            for i in range(N):
                res[i] = [self.arrival[i], self.departure[i]]
            return res
        return arrival_dep_times(N)
                
        
g = Graph()
g.addEdge(0, 1)
g.addEdge(0, 2)
g.addEdge(2, 3)
g.addEdge(2, 4)
g.addEdge(3, 1)
g.addEdge(3, 5)
g.addEdge(4, 5)
g.addEdge(6, 7)

print(g.print_graph())
#print(g.count_vertices())
print(g.arrival_deaparture_times())

defaultdict(<class 'list'>, {0: [1, 2], 2: [3, 4], 3: [1, 5], 4: [5], 6: [7]})
8
{0: [0, 11], 1: [1, 2], 2: [3, 10], 3: [4, 7], 4: [8, 9], 5: [5, 6], 6: [12, 15], 7: [13, 14]}


In [21]:
# topological sorting of a graph:
from collections import defaultdict
class Graph:
    def __init__(self):
        self.graph = defaultdict(list)
    
    def addEdge(self, u, v):
        self.graph[u].append(v)
    
    def print_graph(self):
        return self.graph
    
    def count_vertices(self):
        count = 0
        visit = set()
        for i in self.graph:
            if i not in visit:
                visit.add(i)
                count += 1
                for j in self.graph[i]:
                    if j not in visit:
                        visit.add(j)
                        count += 1
        return count        
    
    def arrival_departure_times(self):
        N = self.count_vertices()
        #print(N)
        def arrival_dep_times(N):
            self.arrival = [None]*N
            self.departure = [None]*N
            self.discovered = [False]*N
            self.time = -1
            def dfs(start):
                self.time += 1
                self.arrival[start] = self.time
                self.discovered[start] = True
                for i in self.graph[start]:
                    if not self.discovered[i]:
                        self.time = dfs(i)
                self.time += 1
                self.departure[start] = self.time
                return self.time 
            for i in range(N):
                if not self.discovered[i]:
                    dfs(i)
            res = {}
            for i in range(N):
                res[i] = [self.arrival[i], self.departure[i]]
            return res
        return arrival_dep_times(N)
    
    def topological_sorting(self):
        d = self.arrival_departure_times()
        print(d)
        departure = [-1]*2*self.count_vertices()
        for i in d:
            departure[d[i][1]] = i
        print(departure)
        for i in departure[::-1]:
            if i !=-1: print(i, end = " ")
                
g = Graph()
g.addEdge(0, 6)
g.addEdge(1, 2)
g.addEdge(1, 4)
g.addEdge(1, 6)
g.addEdge(3, 0)
g.addEdge(3, 4)
g.addEdge(5, 1)
g.addEdge(7, 0)
g.addEdge(7, 1)
print(g.print_graph())
#print(g.count_vertices())
#print(g.arrival_deaparture_times())
print(g.topological_sorting())

defaultdict(<class 'list'>, {0: [6], 1: [2, 4, 6], 3: [0, 4], 5: [1], 7: [0, 1]})
{0: [0, 3], 1: [4, 9], 2: [5, 6], 3: [10, 11], 4: [7, 8], 5: [12, 13], 6: [1, 2], 7: [14, 15]}
[-1, -1, 6, 0, -1, -1, 2, -1, 4, 1, -1, 3, -1, 5, -1, 7]
7 5 3 1 4 2 0 6 None


In [28]:
# kahn's algorithm to get the topological sorting order of the nodes in the graph:

from collections import defaultdict, deque
class Graph:
    def __init__(self):
        self.graph = defaultdict(list)
    
    def addEdge(self, u, v):
        self.graph[u].append(v)
    
    def print_graph(self):
        return self.graph
    
    def count_vertices(self):
        count = 0
        visit = set()
        for i in self.graph:
            if i not in visit:
                visit.add(i)
                count += 1
                for j in self.graph[i]:
                    if j not in visit:
                        visit.add(j)
                        count += 1
        return count 
    def kahn_algorithm(self):
        self.indegree = [0]*self.count_vertices()
        res = []
        for i in self.graph:
            for j in self.graph[i]:
                self.indegree[j]+=1
        s = deque()
        for i in range(len(self.indegree)):
            if self.indegree[i] == 0: s.append(i)
        
        while s:
            node1 = s.pop()
            res.append(node1)
            for node2 in self.graph[node1]:
                self.indegree[node2] -= 1
                if self.indegree[node2] == 0: s.append(node2)
        
        for i in self.indegree:
            if i > 0: return " not a DAG"
        else: return res
                         
    
g = Graph()
g.addEdge(0, 6)
g.addEdge(1, 2)
g.addEdge(1, 4)
g.addEdge(1, 6)
g.addEdge(3, 0)
g.addEdge(3, 4)
g.addEdge(5, 1)
g.addEdge(7, 0)
g.addEdge(7, 1)
print(g.print_graph())
print(g.kahn_algorithm())

defaultdict(<class 'list'>, {0: [6], 1: [2, 4, 6], 3: [0, 4], 5: [1], 7: [0, 1]})
[7, 5, 1, 2, 3, 4, 0, 6]


In [43]:
# Transitive closure of a graph:
from collections import defaultdict
class Graph:
    def __init__(self):
        self.graph = defaultdict(list)
    
    def addEdge(self, u, v):
        self.graph[u].append(v)
    
    def print_graph(self):
        return self.graph
    
    def count_vertices(self):
        count = 0
        visit = set()
        for i in self.graph:
            if i not in visit:
                visit.add(i)
                count += 1
                for j in self.graph[i]:
                    if j not in visit:
                        visit.add(j)
                        count += 1
        return count 
    
    def transitive_closure(self):
        N = self.count_vertices()
        self.res = [[0 for i in range(N)] for row in range(N)]
        # first we need to get the adjacency matrix
        for i in self.graph:
            for j in self.graph[i]:
                self.res[i][j] = 1
        # we need to change the adjacnecy matrix over the iteration of k value --> time complexity O(n^3) and space--> O(n^2)
        for k in range(N):
            for i in range(N):
                for j in range(N):
                    self.res[i][j] = (self.res[i][j] or (self.res[i][k] and self.res[k][j]))
        for i in range(N):
            self.res[i][i] = 1
        return self.res
    
    def transitive_closure_DFS(self):
        N = self.count_vertices()
        self.res = [[0 for i in range(N)] for row in range(N)]
        def DFS_t_c(root, descendant):
            for child in self.graph[descendant]:
                if self.res[root][child] == 0:
                    self.res[root][child] = 1
                    DFS_t_c(root, child)
        for v in range(N):
            self.res[v][v] = 1
            DFS_t_c(v, v)
            
        return self.res
               

g = Graph()
g.addEdge(0, 2)
g.addEdge(1,0)
g.addEdge(3,1)
print(g.print_graph())
print(g.count_vertices())
#print(g.transitive_closure())
print(g.transitive_closure_DFS())

defaultdict(<class 'list'>, {0: [2], 1: [0], 3: [1]})
4
[[1, 0, 1, 0], [1, 1, 1, 0], [0, 0, 1, 0], [1, 1, 1, 1]]


In [70]:
# check if undirected graph contains cycle or not:
from collections import defaultdict, deque
class Graph:
    def __init__(self):
        self.graph = defaultdict(list)
    
    def addEdge(self, u, v):
        self.graph[u].append(v)
    
    def print_graph(self):
        return self.graph
    
    def count_vertices(self):
        count = 0
        visit = set()
        for i in self.graph:
            if i not in visit:
                visit.add(i)
                count += 1
                for j in self.graph[i]:
                    if j not in visit:
                        visit.add(j)
                        count += 1
        return count
    
    def check_cycle_exists_DFS(self):
        N = self.count_vertices()
        self.adjlist = [[] for _ in range(N)]
        for i in self.graph:
            for j in self.graph[i]:
                self.adjlist[i].append(j)
                self.adjlist[j].append(i)
        self.visit = [False]*N
        def DFS_check_cycle(v, parent):
            self.visit[v] = True
            for node in self.graph[v]:
                if not self.visit[node]:
                    if DFS_check_cycle(node, v): return True
                elif node != parent:
                    return True
            return False
        
        if DFS_check_cycle(0, -1): return "There is a cycle present in the graph"
        else: return "There is no cycle present in the graph"

    def check_cycle_exists_BFS(self):
        N = self.count_vertices()
        self.adjlist = [[] for _ in range(N)]
        for i in self.graph:
            for j in self.graph[i]:
                self.adjlist[i].append(j)
                self.adjlist[j].append(i)
        self.visit = [False]*N
        def BFS_check_cycle(start):
            self.visit[start] = True
            q = deque()
            q.append((start, -1))
            while q:
                vertex1, parent = q.popleft()
                for vertex2 in self.graph[vertex1]:
                    if not self.visit[vertex2]:
                        self.visit[vertex2] = True
                        q.append((vertex2, vertex1))
                    elif vertex2 != parent: return True
            return False
        if BFS_check_cycle(0): return "cycle present"
        else: "no cycle"
            
    def check_cycle_graph_coloring(self):
        pass
    
            
        
g = Graph()
g.addEdge(0, 1)
g.addEdge(1, 2)
g.addEdge(2, 4)
g.addEdge(4, 3)
g.addEdge(0, 3)
g.addEdge(0, 4)
print(g.print_graph())
print(g.check_cycle_exists_BFS())

defaultdict(<class 'list'>, {0: [1, 3, 4], 1: [2], 2: [4], 4: [3]})
 cycle present


In [42]:
# check if cycle present in the directed graph: presence of back edge indicates that there is a cycle present in the graph. 
from collections import defaultdict, deque
class Graph:
    def __init__(self):
        self.graph = defaultdict(list)
    
    def add_Edge(self, u, v):
        self.graph[u].append(v)
    
    def print_graph(self):
        return self.graph
    
    def count_vertices(self):
        count = 0
        visit = set()
        for i in self.graph:
            if i not in visit:
                visit.add(i)
                count += 1
                for j in self.graph[i]:
                    if j not in visit:
                        visit.add(j)
                        count += 1
        return count
    def get_all_vertices(self):
        s = set()
        for i in self.graph:
            if i not in s:
                s.add(i)
                for j in self.graph[i]:
                    if j not in s: s.add(j)
        return s
    
    def check_cycle_exists(self):
        """white --> all nodes in initial stage
           gray  --> visited
           black --> visited and processed
        """
        self.white = self.get_all_vertices()
        self.gray = self.black = set()
        def DFS_check_cycle_directed(current):
            self.white.remove(current)
            self.gray.add(current)
            for neighbor in self.graph[current]:
                if neighbor in self.black: continue 
                if neighbor in self.gray: return True
                if DFS_check_cycle_directed(neighbor): return True
            self.gray.remove(current)
            self.black.add(current)
            return False
        while len(self.white)>0:
            current = next(iter(self.white))
            if DFS_check_cycle_directed(current): return True
        return False
"""
white = {2,4,5,6} --> unvisited      self.graph = {1:[2], 4:[1,5], 5:[6], 6:[4]}
gray = {1} --> visited
black = {} --> visited and processed
DFS(4, w, g, b) -> DFS(5, w, g, b) -->DFS(6, w, g, b)
"""
g = Graph()
g.add_Edge(1, 2)
g.add_Edge(4, 1)
g.add_Edge(4, 5)
g.add_Edge(5, 6)
g.add_Edge(6, 4)

print(g.check_cycle_exists())

False


In [60]:
"""
# g.add_Edge(1, 2)
# g.add_Edge(1, 3)
# g.add_Edge(2, 3)
# g.add_Edge(4, 1)
# g.add_Edge(4, 5)
# g.add_Edge(5, 6)
# g.add_Edge(6, 4)
Bi-partite graph: if we can divide it into two sets U and V with every edge having one end point in set U and other set V.

--> a graph is 2-colorable. means parent and child have different colors.
--> a graph is not having odd cycle. odd cycle means, while we doing BFS if we found an edge which connectiong two vertices that
    are of same level. which is not Bipartite graph.

"""
from collections import deque, defaultdict
class Graph:
    def __init__(self):
        self.graph = defaultdict(list)
    def addEdge(self, u, v):
        self.graph[u].append(v)
    def print_graph(self):
        return self.graph
    
    def check_bipartite(self, N):
        self.discovered = [False]*N
        self.level = [None]*N
        def BFS(start):
            self.discovered[start] = True
            self.level[start] = 0
            q = deque()
            q.append(start)
            while q:
                v = q.popleft()
                print(self.level)
                for u in self.graph[v]:
                    if not self.discovered[u]:
                        self.discovered[u] = True
                        self.level[u] = self.level[v]+1
                        q.append(u)
                    elif self.level[u] == self.level[v]: return False
            return True
        if BFS(0): print("BiPartite graph")
        else: print("Not a BiPartite graph")
        
g = Graph()
g.addEdge(0, 3)
g.addEdge(0, 1)
g.addEdge(1, 2)
g.addEdge(2, 3)
g.addEdge(1,0)
g.addEdge(2,1)
g.addEdge(3,2)
g.addEdge(3,0)
# g.addEdge(1,2)
# g.addEdge(2,3)
# g.addEdge(2,8)
# g.addEdge(3,4)
# g.addEdge(4,6)
# g.addEdge(8,9)
# g.addEdge(5,9)
# g.addEdge(5,7)
print(g.print_graph())
print(g.check_bipartite(4))


defaultdict(<class 'list'>, {0: [3, 1], 1: [2, 0], 2: [3, 1], 3: [2, 0]})
[0, None, None, None]
[0, 1, None, 1]
[0, 1, 2, 1]
[0, 1, 2, 1]
BiPartite graph
None


In [10]:
from collections import deque, defaultdict
class Graph:
    def __init__(self):
        self.graph = defaultdict(list)
    def addEdge(self, u, v):
        self.graph[u].append(v)
    def print_graph(self):
        return self.graph
    def check_bipartite_two_colored(self):
        N = self.count_nodes()
        self.visited = [False]*N
        self.colored = [False]*N
        self.visited[0] = True
        def DFS(v):
            for u in self.graph[v]:
                if not self.visited[u]:
                    self.visited[u] = True
                    self.colored[u] = not self.colored[v]
                    if not DFS(u): return False
                elif self.colored[u] == self.colored[v]: return False
            return True
        return True if DFS(0) else False
    def count_nodes(self):
        res = 0
        visit = set()
        q = []
        q.append(0)
        visit.add(0)
        res += 1
        while q:
            node = q.pop(0)
            for i in self.graph[node]:
                if i not in visit:
                    visit.add(i)
                    q.append(i)
                    res += 1
        return res
    
g = Graph()
g.addEdge(0, 1)
g.addEdge(0, 3)
g.addEdge(1, 0)
g.addEdge(1, 2)
g.addEdge(2, 3)
g.addEdge(2, 1)
g.addEdge(3, 0)
g.addEdge(3, 2)
g.addEdge(3, 2)
g.addEdge(0, 2)
g.addEdge(2, 0)

print(g.print_graph())
print(g.count_nodes())
print(g.check_bipartite_two_colored())

defaultdict(<class 'list'>, {0: [1, 3, 2], 1: [0, 2], 2: [3, 1, 0], 3: [0, 2, 2]})
4
False


In [21]:
#Determine if an undirected graph is a Tree (Acyclic Connected Graph)
from collections import defaultdict, deque
class Graph:
    def __init__(self):
        self.graph = defaultdict(list)
    
    def addEdge(self, u, v):
        self.graph[u].append(v)
    
    def print_graph(self):
        return self.graph
    
    def count_vertices(self):
        count = 0
        visit = set()
        for i in self.graph:
            if i not in visit:
                visit.add(i)
                count += 1
            for j in self.graph[i]:
                if j not in visit:
                    visit.add(j)
                    count += 1
        return count
    def check_undirected_graph_is_tree(self):
        N = self.count_vertices()
        self.visited = [False]*N
        self.adjlist = [[] for _ in range(N)]
        for i in self.graph:
            for j in self.graph[i]:
                self.adjlist[i].append(j)
                self.adjlist[j].append(i)
        def DFS(vertex, parent):
            self.visited[vertex] = True
            for w in self.adjlist[vertex]:
                if not self.visited[w]:
                    if not DFS(w, vertex): return False
                elif w != parent: return False
            return True
        return DFS(0,-1)
    
        
g = Graph()
g.addEdge(0,1)
g.addEdge(1,2)
g.addEdge(2,3)
g.addEdge(3,4)
g.addEdge(4,5)
g.addEdge(5,0)
print(g.count_vertices())
print(g.check_undirected_graph_is_tree())

6
False


In [5]:
# Total number of paths in given digraph from given source to destination having exactly m edges
from collections import defaultdict, deque
class Graph:
    def __init__(self):
        self.graph = defaultdict(list)
    
    def addEdge(self, u, v):
        self.graph[u].append(v)
    
    def print_graph(self):
        return self.graph
    
    def count_vertices(self):
        count = 0
        visit = set()
        for i in self.graph:
            if i not in visit:
                visit.add(i)
                count += 1
                for j in self.graph[i]:
                    if j not in visit:
                        visit.add(j)
                        count += 1
        return count
    
    def paths_with_m_edges(self, src, dst, m):
        q = deque()
        q.append((src, 0)) #enque the current vertex and current depth
        count = 0
        while q:
            vertex, depth = q.popleft()
            if vertex == dst and depth == m:
                count += 1
            if depth > m: break 
            for node in self.graph[vertex]:
                q.append((node, depth+1))   
        return count
    
g = Graph()
g.addEdge(0,6)
g.addEdge(0,1)
g.addEdge(1,6)
g.addEdge(1,5)
g.addEdge(1,2)
g.addEdge(2,3)
g.addEdge(3,4)
g.addEdge(5,2)
g.addEdge(5,3)
g.addEdge(5,4)
g.addEdge(6,5)
g.addEdge(7,6)
g.addEdge(7,1)
print(g.paths_with_m_edges(0,3,4))

3


In [15]:
# check whether there is 2edge connectivity in the graph:
from collections import defaultdict, deque
class Graph:
    def __init__(self):
        self.graph = defaultdict(list)
    
    def addEdge(self, u, v):
        self.graph[u].append(v)
    
    def print_graph(self):
        return self.graph
    
    def count_vertices(self):
        count = 0
        visit = set()
        for i in self.graph:
            if i not in visit:
                visit.add(i)
                count += 1
            for j in self.graph[i]:
                if j not in visit:
                    visit.add(j)
                    count += 1
        return count
    """ K-Edge connectivity: for two vertices v and w are are said to be k-connected, if we remove upto k-1 edges, those 
        vertives v and w are still have connected. 
        eg: for 2 connectivity even if we remove an edge, there should be presence of all node while we traverse with DFS or BFS
            for 2 connected, if there is an edge present then it's not 2 edge connected graph.
            
       A graph is said to be k-Edge connected iff every two vertices are k-Edge connected. Applications of connectivity is
       to find out the fault tolerance of network: how many connections can fail without cutting of communication"""
    def check_two_edge_connectivity(self): # this algorithm here is trying to find out the bridges
        N = self.count_vertices()
        self.adjlist = [[] for _ in range(N)]
        for i in self.graph:
            for j in self.graph[i]:
                self.adjlist[i].append(j)
                self.adjlist[j].append(i)
        self.visit = [False]*N
        self.arrival = [None]*N
        self.time = 0
        def DFS(vertex, parent):
            self.time += 1
            self.arrival[vertex] = self.time
            self.visit[vertex] = True
            A =  self.arrival[vertex]
            for neighbor in self.adjlist[vertex]:
                if not self.visit[neighbor]:
                    A = min(A, DFS(neighbor, vertex))
                elif neighbor != parent:
                    A = min(A, self.arrival[neighbor])
            if A < self.arrival[vertex] and parent != -1:
                print((parent, vertex), end = " ")
            return A
        DFS(0, -1)
    
g = Graph()
g.addEdge(0,2)
g.addEdge(1,2)
g.addEdge(2,3)
g.addEdge(2,4)
g.addEdge(3,4)
g.addEdge(3,5)
print(g.check_two_edge_connectivity())

(3, 4) (2, 3) None


In [17]:
# check if given graph is directed acyclic graph or not(DAG)
from collections import defaultdict, deque
class Graph:
    def __init__(self):
        self.graph = defaultdict(list)
    
    def addEdge(self, u, v):
        self.graph[u].append(v)
    
    def print_graph(self):
        return self.graph
    def count_vertices(self):
        count = 0
        visit = set()
        for i in self.graph:
            if i not in visit:
                visit.add(i)
                count += 1
            for j in self.graph[i]:
                if j not in visit:
                    visit.add(j)
                    count += 1
        return count
    def check_DAG(self): # check if there is any back edge present in graph: departure[u]<departure[v]
        N = self.count_vertices()
        print(N)
        self.visit = [False]*N
        self.departure = [None]*N
        self.arrival = [None]*N
        def DFS(v, time):
            time += 1
            self.visit[v] = True
            self.arrival[v] = time
            for u in self.graph[v]:
                if not self.visit[u]:
                    time = DFS(u, time)
            time = time + 1
            self.departure[v] = time
            return time    
         
        time = -1
        for i in range(N):
            if not self.visit[i]:
                time = DFS(i, time)
        print(self.departure)
        print(self.arrival)
        for u in range(N):
            for v in self.graph[u]:
                if self.departure[u]<=self.departure[v]: return False
        return True
        
    
g = Graph()
g.addEdge(0,1)
g.addEdge(0,3)
g.addEdge(1,2)
g.addEdge(1,3)
g.addEdge(3,2)
g.addEdge(3,4)
g.addEdge(0,3)
g.addEdge(5,6)
g.addEdge(6,3)
print(g.print_graph())
print(g.check_DAG())

defaultdict(<class 'list'>, {0: [1, 3, 3], 1: [2, 3], 3: [2, 4], 5: [6], 6: [3]})
7
[9, 8, 3, 7, 6, 13, 12]
[0, 1, 2, 4, 5, 10, 11]
True


In [24]:
# Disjoint set data structure algorithm: Union Find algorithm
class DisjointSet:
    parent = {}
    def makeSet(self, universe):
        for i in universe:
            self.parent[i] = i
    
    def Find(self, k):
        if self.parent[k] == k: return k
        return self.Find(self.parent[k])
    
    def Union(self, a, b):
        x = self.Find(a)
        y = self.Find(b)
        self.parent[x] = y
    
def printSets(universe, ds):
    print([ds.Find(i) for i in universe])
        
    
universe = [1,2,3,4,5]
ds = DisjointSet()
ds.makeSet(universe)
printSets(universe, ds)

ds.Union(4,3)
printSets(universe, ds)

ds.Union(2,1)
printSets(universe, ds)

ds.Union(1,3)
printSets(universe, ds)

[1, 2, 3, 4, 5]
[1, 2, 3, 3, 5]
[1, 1, 3, 3, 5]
[3, 3, 3, 3, 5]


In [26]:
# improved version of disjoint sets:
class DisjointSet:
    parent = {}
    rank = {}
    def makeSet(self, universe):
        for i in universe:
            self.parent[i] = i
            self.rank[i] = 0
    
    def Find(self, k):
        if self.parent[k] == k: return k
        return self.Find(self.parent[k])
    
    def Union(self, a, b):
        x = self.Find(a)
        y = self.Find(b)
        if x == y:
            return 
        
        if self.rank[x]>self.rank[y]:
            self.parent[y] = x
        elif self.rank[x]<self.rank[y]:
            self.parent[x] = y
        else:
            self.parent[x] = y
            self.rank[y] = self.rank[y]+1
        self.parent[x] = y
    
def printSets(universe, ds):
    print([ds.Find(i) for i in universe])
        
universe = [1,2,3,4,5]
ds = DisjointSet()
ds.makeSet(universe)
printSets(universe, ds)

ds.Union(4,3)
printSets(universe, ds)

ds.Union(2,1)
printSets(universe, ds)

ds.Union(1,3)
printSets(universe, ds)

[1, 2, 3, 4, 5]
[1, 2, 3, 3, 5]
[1, 1, 3, 3, 5]
[3, 3, 3, 3, 5]


In [5]:
# snake and ladder problem
def snake_ladder(matrix):
    n = len(matrix)
    q = [1]
    visited = [0]*((n*n)+1)
    moves, mini = 0, n*n
    while q:
        for _ in range(len(q)):
            current = q.pop(0)
            if current == n*n: mini = min(moves, mini)
            for i in range(1, 7):
                num = current + i
                if num > n*n: break
                if visited[num] == 0:
                    visited[num] = 1
                    row = (n-(num-1)//n)-1
                    if (n-row) % 2 != 0: col = (num-1)%n
                    else: col = (n - (num-1)% n)-1
                        
                    if matrix[row][col] == -1: q.append(num)
                    else: q.append(matrix[row][col])
        moves += 1
    return -1 if mini == n*n else mini

m = [[-1,-1,-1,-1,-1,-1],
     [-1,-1,-1,-1,-1,-1],
     [-1,-1,-1,-1,-1,-1],
     [-1,35,-1,-1,13,-1],
     [-1,-1,-1,-1,-1,-1],
     [-1,15,-1,-1,-1,-1]]   
snake_ladder(m)

4

In [1]:
# algoexpert - river sizes:
def river_sizes(matrix):
    res = []
    visited = [[False for value in row] for row in matrix]
    for i in range(len(matrix)):
        for j in range(len(matrix[i])):
            if visited[i][j] == True:
                continue
            traverse_node(i, j, matrix, res, visited)
    return res

def traverse_node(i, j, matrix, res, visited):
    current_size = 0
    nodestoexplore = [[i, j]]
    while nodestoexplore:
        current = nodestoexplore.pop()
        i, j = current[0], current[1]
        if visited[i][j]: continue
        visited[i][j] = True
        if matrix[i][j] == 0: continue
        current_size += 1
        unvisited = get_unvisited(i, j, matrix, visited)
        for node in unvisited:
            nodestoexplore.append(node)
    if current_size > 0: res.append(current_size)
        
def get_unvisited(i, j, matrix, visited):
    arr = []
    if i > 0 and not visited[i-1][j]: arr.append([i-1, j]) # up
    if i < len(matrix)-1 and not visited[i+1][j]: arr.append([i+1, j]) # down
    if j > 0 and not visited[i][j-1]: arr.append([i, j-1]) # left
    if j < len(matrix[i])-1 and not visited[i][j+1]: arr.append([i, j+1]) # right
    return arr
    
    
matrix = [[1,0,0,1,0],
          [1,0,1,0,0],
          [0,0,1,0,1],
          [1,0,1,0,1],
          [1,0,1,1,0]]
river_sizes(matrix)

[2, 1, 5, 2, 2]

In [1]:
# leetcode 542 -> 01 matrix
def updateMatrix(matrix):
    queue = []
    visited = [[0 for i in row] for row in matrix]
    m, n = len(matrix), len(matrix[0])
    for i in range(m):
        for j in range(n):
            if matrix[i][j] == 0:
                visited[i][j] = 1
                queue.append([i, j])
    print(queue, visited)
    dirs = [[0,1], [0,-1], [1, 0], [-1, 0]]
    while queue:
        p = queue.pop(0)
        currentx = p[0]
        currenty = p[1]
        for i in dirs:
            x = currentx + i[0]
            y = currenty + i[1]
            if x < 0 or x >len(matrix)-1 or y < 0 or y > len(matrix[0])-1 or visited[x][y]:
                continue
            matrix[x][y] = matrix[currentx][currenty] + 1
            visited[x][y] = 1
            queue.append([x, y])
    return matrix
        
m = [[0,0,0],
     [0,1,0],
     [1,1,1]]
updateMatrix(m)

[[0, 0], [0, 1], [0, 2], [1, 0], [1, 2]] [[1, 1, 1], [1, 0, 1], [0, 0, 0]]


[[0, 0, 0], [0, 1, 0], [1, 2, 1]]

In [4]:
# amazon's OA question:
def demolition_robot(matrix):
    queue = []
    visited = [[0 for i in row] for row in matrix]
    m,n = len(matrix), len(matrix[0])
    for i in range(m):
        for j in range(n):
            if matrix[i][j] == 0:
                visited[i][j] = 1
                queue.append([i, j])
    print(queue, visited)
    dirs = [[0,1], [0,-1], [1,0],[-1,0]]
    while queue:
        p = queue.pop(0)
        currentx = p[0]
        currenty = p[1]
        for i in dirs:
            x, y = currentx+i[0], currenty+i[1]
            if x<0 or x>len(matrix)-1 or y < 0 or y > len(matrix[0])-1 or visited[x][y]: continue
            matrix[x][y] = matrix[currentx][currenty]+1
            visited[x][y] = 1
            queue.append([x,y])
    return matrix

matrix = [[1,0,0],[1,0,0],[1,1,1]]
demolition_robot(matrix)

[[0, 1], [0, 2], [1, 1], [1, 2]] [[0, 1, 1], [0, 1, 1], [0, 0, 0]]


[[1, 0, 0], [1, 0, 0], [2, 1, 1]]

In [5]:
## grpah problems DFS traversals:

# 1. river sizes:
def river_sizes(matrix):
    if not matrix: return []
    res = []
    for i in range(len(matrix)):
        for j in range(len(matrix[i])):
            if matrix[i][j] == 1:
                res.append(dfs(i, j, matrix))
    return res

def dfs(i, j, grid):
    if i < 0 or i >= len(grid) or j < 0 or j >= len(grid[0]) or grid[i][j] == 0: return 0
    grid[i][j] = 0
    return 1 + dfs(i-1, j, grid) + dfs(i+1, j, grid) + dfs(i, j-1, grid) + dfs(i, j+1, grid)
    
matrix = [[1,0,0,1,0],
          [1,0,1,0,0],
          [0,0,1,0,1],
          [1,0,1,0,1],
          [1,0,1,1,0]]
river_sizes(matrix)

[2, 1, 5, 2, 2]

In [6]:
# leetcode 329: longest increasing path in matrix
def longest_increasing_path(matrix):
    dirs = [[0,1], [1, 0], [-1,0], [0,-1]]
    if len(matrix) == 0: return 0
    res = 0
    cache = [[0 for i in row] for row in matrix]
    for i in range(len(matrix)):
        for j in range(len(matrix[i])):
            longest = get_longest(matrix, cache, i, j, dirs)
            res = max(res, longest)
    return res

def get_longest(matrix, cache, i, j, dirs):
    maxi = 0
    if cache[i][j] > 0: return cache[i][j]
    for direction in dirs:
        x,y = i+direction[0], j+direction[1]
        if x >= 0 and x<len(matrix) and y>=0 and y<len(matrix[0]) and matrix[x][y] > matrix[i][j]:
            longest = get_longest(matrix, cache, x, y, dirs)
            maxi = max(maxi, longest)
    cache[i][j] = maxi + 1
    
    return cache[i][j]
            
            
m = [[9,9,4],[6,6,8],[2,1,1]]
longest_increasing_path(m)

4

In [8]:
# chess knigt problem: find shortest path from source to destination.
"""
N --> size of board
start --> (N-1, 0) 
destination --> (0, N-1)
"""
from collections import deque
def min_steps(N):
    row = [2, 2, -2, -2, 1, 1, -1, -1]
    col = [-1, 1, 1, -1, 2, -2, 2, -2]
    visited = set()
    q = deque()
    src, dst = (N-1, 0, 0), (0, N-1)
    q.append(src)
    while q:
        node = q.popleft()
        x, y, dist= node
        if x == dst[0] and y == dst[1]:
            return dist
 
        if node not in visited:
            visited.add(node)
            for i in range(8):
                x1 = x + row[i]
                y1 = y + col[i]
 
                if is_valid(x1, y1, N):
                    q.append((x1, y1, dist+1))
    return float('inf')
  
def is_valid(x, y, n):
    return not (x<0 or y<0 or x>=n or y>=n)


N = 8
min_steps(N)
    

6

In [20]:
# check if given graph is strongly connected or not:
""" given directed graph is strongly connected if and only if there is a possibility to reach all other verteces from
    every other vertex"""
from collections import defaultdict
class Graph:
    def __init__(self):
        self.graph = defaultdict(list)
        self.reversed_graph = defaultdict(list)
    def addEdge(self, u, v):
        self.graph[u].append(v)
        self.reversed_graph[v].append(u)
    def print_graph(self):
        print(self.reversed_graph)
        return self.graph
    def count_vertices(self):
        count = 0
        visit = set()
        for i in self.graph:
            if i not in visit:
                visit.add(i)
                count += 1
            for j in self.graph[i]:
                if j not in visit:
                    visit.add(j)
                    count += 1
        return count
    
    def check_strongly_connected_first(self):
        # O(V(V+E)) Time complexity
        N = self.count_vertices()
        
        def DFS(v, visited):
            visited[v] = True
            for u in self.graph[v]:
                if not visited[u]:
                    DFS(u, visited)
                
        for i in range(N):
            visited = [False]*N
            DFS(i, visited)
            for i in visited:
                if not i: return False
        return True
    def check_strongly_connected_second(self):
        # O(V+E)
        N = self.count_vertices()
        visited = [False]*N
        self.DFS_second(self.graph, 0, visited) # choose any random vertex, here i'm choosing 0
        for i in visited:
            if not i: return False
        visited = [False]*N
        self.DFS_second(self.reversed_graph, 0, visited)
        for i in visited:
            if not i: return False
        return True
    def DFS_second(self,graph, v, visited):
        visited[v] = True
        for u in graph[v]:
            if not visited[u]:
                self.DFS_second(graph, u, visited)
                
g = Graph()
g.addEdge(0,4)
g.addEdge(1,0)
g.addEdge(1,2)
g.addEdge(2,1)
g.addEdge(2,4)
g.addEdge(3,1)
g.addEdge(3,2)
g.addEdge(4,3)
print(g.print_graph())
print(g.check_strongly_connected_first())
print(g.check_strongly_connected_second())

defaultdict(<class 'list'>, {4: [0, 2], 0: [1], 2: [1, 3], 1: [2, 3], 3: [4]})
defaultdict(<class 'list'>, {0: [4], 1: [0, 2], 2: [1, 4], 3: [1, 2], 4: [3]})
True
True


In [22]:
# check if given graph is strongly connected or not using ONE DFS traversal:
from collections import defaultdict
class Graph:
    def __init__(self):
        self.graph = defaultdict(list)
        self.reversed_graph = defaultdict(list)
    def addEdge(self, u, v):
        self.graph[u].append(v)
        self.reversed_graph[v].append(u)
    def print_graph(self):
        print(self.reversed_graph)
        return self.graph
    def count_vertices(self):
        count = 0
        visit = set()
        for i in self.graph:
            if i not in visit:
                visit.add(i)
                count += 1
            for j in self.graph[i]:
                if j not in visit:
                    visit.add(j)
                    count += 1
        return count
    def check_strongly_connected_one_DFS(self):
        N = self.count_vertices()
        self.visited = [False]*N
        self.arrival = [None]*N
        is_scc = True
        time = -1
        def DFS(v, is_scc, time):
            if not is_scc: return 0, is_scc, time
            time = time+1
            self.arrival[v] = time
            self.visited[v] = True
            arr = self.arrival[v]
            
            for w in self.graph[v]:
                if not self.visited[w]:
                    arr = min(arr, DFS(w, is_scc, time)[0])
                else:
                    arr = min(arr, self.arrival[w])
            if v and arr == self.arrival[v]: is_scc = False
            return arr, is_scc, time
        is_scc = DFS(0, is_scc, time)[1]
        print(self.visited)
        for i in range(N):
            if not self.visited[i]:
                is_scc = False
        return is_scc
g = Graph()
g.addEdge(0,4)
g.addEdge(1,0)
g.addEdge(1,2)
g.addEdge(2,1)
g.addEdge(2,4)
g.addEdge(3,1)
g.addEdge(3,2)
g.addEdge(4,3)
print(g.print_graph())
print(g.check_strongly_connected_one_DFS())

defaultdict(<class 'list'>, {4: [0, 2], 0: [1], 2: [1, 3], 1: [2, 3], 3: [4]})
defaultdict(<class 'list'>, {0: [4], 1: [0, 2], 2: [1, 4], 3: [1, 2], 4: [3]})
[True, True, True, True, True]
True


In [21]:
#Union find algorithm for cycle detection in  graph: *************
"""
1. create the disjoint sets for each of the vertex in the graph
2. for every edge u-->v in the graph:
   --> find the root of the sets to which elements u and v belongs to.
   --> if both u and v have same root in disjoint sets, a cycle is found

"""

from collections import defaultdict
class Graph:
    def __init__(self):
        self.graph = defaultdict(list)
        #self.reversed_graph = defaultdict(list)
    def addEdge(self, u, v):
        self.graph[u].append(v)
        #self.graph[v].append(u)
    def print_graph(self):
        return self.graph
    def count_vertices(self):
        count = 0
        visit = set()
        for i in self.graph:
            if i not in visit:
                visit.add(i)
                count += 1
            for j in self.graph[i]:
                if j not in visit:
                    visit.add(j)
                    count += 1
        return count
    
    def check_cycle_exists(self):
        ds = DisjointSet()
        N = self.count_vertices()
        ds.make_set(N)
        for u in range(N):
            for v in self.graph[u]:
                x = ds.Find(u)
                y = ds.Find(v)
                if x == y: return True
                else: ds.Union(x,y)
        return False
    
class DisjointSet:
    parent = {}
    g = Graph()
    N = g.count_vertices()
    def make_set(self,N):
        for i in range(N):
            self.parent[i] = i
    
    def Find(self, k):
        if self.parent[k] == k:
            return k
        return self.Find(self.parent[k])
    
    def Union(self, a,b):
        x, y = self.Find(a), self.Find(b)
        self.parent[x] = y
        
              
g = Graph()
g.addEdge(0,1)
g.addEdge(1,3)
g.addEdge(2,3)
#g.addEdge(0,2)
g.addEdge(1,4)
g.addEdge(6,4)
g.addEdge(4,5)
g.addEdge(6,7)
#g.addEdge(5,7)
print(g.count_vertices())
print(g.check_cycle_exists())

8
False


In [7]:
""" minimum spanning tree: spanning tree of a connected undirected graph, it connects all the vertices together with minimal
    total weighting for it's edges. """
# kruskal's algorithm for finding minimum spanning tree:
from collections import defaultdict
class DisjointSet:
    parent = {}
#     g = Graph
#     N = g.self.count_vertices()
    def make_set(self, N):
        for i in range(N):
            self.parent[i] = i
    
    def Find(self, k):
        if self.parent[k] == k: return k
        return self.parent[self.parent[k]]
    
    def Union(self, a, b):
        x, y = self.Find(a), self.Find(b)
        self.parent[x] = y
def kruskal_algorithm(edges, N):
    edges.sort(key = lambda x: x[2])
    MST = []
    ds = DisjointSet()
    ds.make_set(N)
    i = 0
    while len(MST) != N-1: #for n number of vertices there could be atmost n-1 edges
        (src, dst, weight) = edges[i]
        i+=1
        x, y = ds.Find(src), ds.Find(dst)
        if x != y: # if both src and dst have different parents, add to mst and perform union operation
            MST.append((src, dst, weight))
            ds.Union(y,x)
            
    return MST
    
edges = [(0, 1, 7),(1, 2, 8),(0, 3, 5),(1, 3, 9),(1, 4, 7), (2, 4, 5),(3, 4, 15),(3, 5, 6),(4, 5, 8),(4, 6, 9),(5, 6, 11)]
N = 7
kruskal_algorithm(edges, N)
    

[(0, 3, 5), (2, 4, 5), (3, 5, 6), (0, 1, 7), (1, 4, 7), (4, 6, 9)]

In [2]:
# Single-Source Shortest Paths – Dijkstra’s Algorithm
# for negative weighted edges, it may give the results well and it may not.
# for undirected graphs also we can do by converting it into to the directed graph. for unweighted graphs, the edge weight
# considering as 1.
from collections import defaultdict
import heapq   
class Graph:
    def __init__(self):
        self.graph = defaultdict(list)
         
    def addEdge(self, u, v, weight):
        self.graph[u].append((v, weight))
        
    def print_graph(self):
        return self.graph
    def count_vertices(self):
        count = 0
        visit = set()
        for i in self.graph:
            if i not in visit:
                visit.add(i)
                count += 1
            for j in self.graph[i]:
                if j[0] not in visit:
                    visit.add(j[0])
                    count += 1
        return count
    
    def shortest_paths_dijkstra(self, source):
        N = self.count_vertices()
        pq = []
        heapq.heapify(pq)
        heapq.heappush(pq, (0, source)) # initially source is at distacnce 0 to itself
        dist = [float('inf')]*N
        dist[source] = 0
        done = [False]*N
        done[source] = True
        print(pq)
        prev = [-1]*N
        while pq:
            u = heapq.heappop(pq)[1]
            for edge in self.graph[u]:
                v, weight = edge
                if not done[v] or (dist[u]+weight) < dist[v]:
                    dist[v] = dist[u]+weight
                    prev[v] = u
                    heapq.heappush(pq, (dist[v], v))
            done[u] = True
        return dist
    
    def shortest_paths2(self, source):
        pass
          
g = Graph()
g.addEdge(0,1,10)
g.addEdge(0,4,3)
g.addEdge(1,2,2)
g.addEdge(1,4,4)
g.addEdge(4,1,1)
g.addEdge(4,2,8)
g.addEdge(4,3,2)
g.addEdge(3,2,7)
g.addEdge(2,3,9)
print(g.print_graph())
#print(g.count_vertices())
print(g.shortest_paths_dijkstra(0))

defaultdict(<class 'list'>, {0: [(1, 10), (4, 3)], 1: [(2, 2), (4, 4)], 4: [(1, 1), (2, 8), (3, 2)], 3: [(2, 7)], 2: [(3, 9)]})
[(0, 0)]
[0, 4, 6, 5, 3]


In [4]:
# for unweighted  and undirected graph dijkstra's algorithm

from collections import defaultdict
class Graph:
    def __init__(self):
        self.graph = defaultdict(list)
    
    def addEdge(self, u, v):
        self.graph[u].append(v)
        self.graph[v].append(u)
        
    def print_graph(self):
        return self.graph
    
    def count_vertices(self):
        count = 0
        visit = set()
        for i in self.graph:
            if i not in visit:
                visit.add(i)
                count += 1
            for j in self.graph[i]:
                if j[0] not in visit:
                    visit.add(j[0])
                    count += 1
        return count
    
    def single_source_shortest_paths(self, source):
        q = []
        visited = {}
        for neighbour in self.graph[source]:
            
    
    
g = Graph()
g.addEdge(0, 1)
g.addEdge(0, 2)
g.addEdge(2, 3)
g.addEdge(2, 4)
g.addEdge(3, 1)
g.addEdge(3, 5)
g.addEdge(4, 5)
g.addEdge(6, 7)
print(g.print_graph())

defaultdict(<class 'list'>, {0: [1, 2], 1: [0, 3], 2: [0, 3, 4], 3: [2, 1, 5], 4: [2, 5], 5: [3, 4], 6: [7], 7: [6]})


In [27]:
# Single-Source Shortest Paths – Bellman Ford Algorithm, it can handle well in the case of graph has negative edge weights
# the limitation of Bellman ford algorithm is if there is negative cycle(total weight of cycle is negative), it wont work.
# time complexity O(VE)
from collections import defaultdict
import heapq   
class Graph:
    def __init__(self):
        self.graph = defaultdict(list)
        self.edges = []
    def addEdge(self, u, v, weight):
        self.graph[u].append((v, weight))
        self.edges.append((u, v, weight))
        
    def print_graph(self):
        return self.graph, self.edges
    
    def count_vertices(self):
        count = 0
        visit = set()
        for i in self.graph:
            if i not in visit:
                visit.add(i)
                count += 1
            for j in self.graph[i]:
                if j[0] not in visit:
                    visit.add(j[0])
                    count += 1
        return count
    def BellmanFord(self, source):
        N = self.count_vertices()
        dist = [float('inf')]*N
        prev = [-1]*N
        dist[source] = 0
        for k in range(N-1): # relaxation has to done in N-1 times
            for (u, v, w) in self.edges:
                if dist[u]+w < dist[v]:
                    dist[v] = dist[u]+w
                    prev[v] = u
                    
        # so far relaxation has done for n-1 times, now run for nth time as well
        for (u, v, w) in self.edges:
            if dist[u]+w < dist[v]:
                return "Negetive edge cycle found"
            else: return dist
    
g = Graph()
g.addEdge(0,1,-1)
g.addEdge(0,2,4)
g.addEdge(1,2,3)
g.addEdge(1,3,2)
g.addEdge(1,4,2)
g.addEdge(3,2,5)
g.addEdge(3,1,1)
g.addEdge(4,3,-3)
print(g.count_vertices())
#print(g.print_graph())
print(g.BellmanFord(0))

5
[0, -1, 2, -2, 1]


In [20]:
# all paires shortest paths - Floyd warshall's algorithm
from collections import defaultdict
class Graph:
    def __init__(self):
        self.graph = defaultdict(list)
        
    def addEdge(self, u, v, weight):
        self.graph[u].append((v, weight))
        
        
    def print_graph(self):
        return self.graph
    
    def count_vertices(self):
        count = 0
        visit = set()
        for i in self.graph:
            if i not in visit:
                visit.add(i)
                count += 1
            for j in self.graph[i]:
                if j[0] not in visit:
                    visit.add(j[0])
                    count += 1
        return count
    
    def floydWarshal(self):
        N = self.count_vertices()
        print(N)
        adjMatrix = [[float('inf') for i in range(N)]for j in range(N)]
        for i in range(len(adjMatrix)):
            for j in range(len(adjMatrix[0])):
                if i ==j: adjMatrix[i][j] = 0
              
        for i in self.graph:
            for j in self.graph[i]:
                adjMatrix[i][j[0]] = j[1]
        print(adjMatrix)
        cost = adjMatrix.copy()
        path = [[None for x in range(N)] for y in range(N)]
        
        for v in range(N):
            for u in range(N):
                if v == u: path[v][u] = 0
                elif cost[v][u] != float('inf'):
                    path[v][u] = v
                else:
                    path[v][u] = -1
        print(path)
        # now run floyd warshall
        for k in range(N):
            for v in range(N):
                for u in range(N):
                    if cost[v][k] != float('inf') and cost[k][u] != float('inf') and cost[v][k]+cost[k][u]<cost[v][u]:
                        cost[v][u] = cost[v][k]+cost[k][u]
                        path[v][u] = path[k][u]
                
                if cost[v][v] < 0:
                    print("negative cycle found")
                    return 
        print(cost, path)
        self.print_solution(path, N)
    
    def print_solution(self,path, n):
        for v in range(n):
            for u in range(n):
                if u != v and path[v][u] != -1:
                    print(f"shortest path from {v} -> {u} is ({v}", end = " ") 
                    self.print_path(path, v, u)
                    print(f"{u})")
                    
    def print_path(self, path, v, u):
        if path[v][u] == v:
            return 
        
        self.print_path(path, v, path[v][u])
        print(path[v][u], end = " ")
                
       
g = Graph()
g.addEdge(0,2,-2)
g.addEdge(1,0,4)
g.addEdge(3,1,-1)
g.addEdge(2,3,2)
g.addEdge(1,2,3)
#print(g.print_graph())
print(g.floydWarshal())

4
[[0, inf, -2, inf], [4, 0, 3, inf], [inf, inf, 0, 2], [inf, -1, inf, 0]]
[[0, -1, 0, -1], [1, 0, 1, -1], [-1, -1, 0, 2], [-1, 3, -1, 0]]
[[0, -1, -2, 0], [4, 0, 2, 4], [5, 1, 0, 2], [3, -1, 1, 0]] [[0, 3, 0, 2], [1, 0, 0, 2], [1, 3, 0, 2], [1, 3, 0, 0]]
shortest path from 0 -> 1 is (0 2 3 1)
shortest path from 0 -> 2 is (0 2)
shortest path from 0 -> 3 is (0 2 3)
shortest path from 1 -> 0 is (1 0)
shortest path from 1 -> 2 is (1 0 2)
shortest path from 1 -> 3 is (1 0 2 3)
shortest path from 2 -> 0 is (2 3 1 0)
shortest path from 2 -> 1 is (2 3 1)
shortest path from 2 -> 3 is (2 3)
shortest path from 3 -> 0 is (3 1 0)
shortest path from 3 -> 1 is (3 1)
shortest path from 3 -> 2 is (3 1 0 2)
None


In [33]:
# Find Cost of Shortest Path in DAG using one pass of Bellman-Ford
from collections import defaultdict
class Graph:
    def __init__(self):
        self.graph = defaultdict(list)
        
    def addEdge(self, u, v, weight):
        self.graph[u].append((v, weight))
        
        
    def print_graph(self):
        return self.graph
    
    def count_vertices(self):
        count = 0
        visit = set()
        for i in self.graph:
            if i not in visit:
                visit.add(i)
                count += 1
            for j in self.graph[i]:
                if j[0] not in visit:
                    visit.add(j[0])
                    count += 1
        return count
    def findShortestDistance(self,source):
        N = self.count_vertices()
        self.departure = [-1]*N
        self.discovered = [False]*N
        self.time = 0
        def dfs(v):
            self.discovered[v] = True
            for i in self.graph[v]:
                u = i[0]
                if not self.discovered[u]:
                    self.time = dfs(u)
                    
            self.departure[v] = self.time
            self.time += 1
            return self.time
        
        for i in range(N):
            if not self.discovered[i]:
                self.time = dfs(i)
        
        cost = [float('inf')]*N
        cost[source] = 0
        
        for i in reversed(range(N)):
            v = self.departure[i]
            for e in self.graph[v]:
                u, w = e
                if cost[v] != float('inf') and cost[v]+w < cost[u]:
                    cost[u] = cost[v]+w
        print(self.departure)
        for i in range(N-1):
            print(f"dist({source},{i}) = {cost[i]}")           
         
g = Graph()
g.addEdge(0,6,2)
g.addEdge(1,2,-4)
g.addEdge(1,4,1)
g.addEdge(1,6,8)
g.addEdge(3,0,3)
g.addEdge(3,4,5)
g.addEdge(5,1,2)
g.addEdge(7,0,6)
g.addEdge(7,1,-1)
g.addEdge(7,3,4)
g.addEdge(7,5,-4)
print(g.print_graph())
print(g.count_vertices())
print(g.findShortestDistance(7))

defaultdict(<class 'list'>, {0: [(6, 2)], 1: [(2, -4), (4, 1), (6, 8)], 3: [(0, 3), (4, 5)], 5: [(1, 2)], 7: [(0, 6), (1, -1), (3, 4), (5, -4)]})
8
[1, 4, 2, 5, 3, 6, 0, 7]
dist(7,0) = 6
dist(7,1) = -2
dist(7,2) = -6
dist(7,3) = 4
dist(7,4) = -1
dist(7,5) = -4
dist(7,6) = 6
None


In [26]:
# Least Cost Path in Weighted Digraph using BFS
from collections import deque, defaultdict
class Graph:
    def __init__(self, N, edges, x):
        self.adjList = [[] for _ in range(3*N)]
        for v, u, weight in edges:
            if weight == 3*x:
                self.adjList[v].append(v+N)
                self.adjList[v+N].append(v+2*N)
                self.adjList[v+2*N].append(u)
            elif weight == 2*x:
                self.adjList[v].append(v+N)
                self.adjList[v+N].append(u)
            else:
                self.adjList[v].append(u)
        print(self.adjList)
        
def BFS(graph, source, dest):
    discovered = [False]*3*N
    discovered[source] = True
    predecessor = [-1]*3*N
    q = deque()
    q.append(source)
    while q:
        current = q.popleft()
        if current == dest:
            print("least cost path between", source, " and ", dest, "is ", end = " ")
            print("having cost", print_path(predecessor, dest, -1, N))
            
        for v in graph.adjList[current]:
            if not discovered[v]:
                discovered[v] = True
                q.append(v)
                predecessor[v] = current
def print_path(arr, v, cost, n):
    if v>=0:
        cost = print_path(arr, arr[v], cost, n)
        cost += 1
        if v<n:
            print(v, end = ' ')
    return cost
                
if __name__ == '__main__':
    x = 1
    edges = [(0, 1, 3 * x), (0, 4, 1 * x), (1, 2, 1 * x), (1, 3, 3 * x),(1, 4, 1 * x), (4, 2, 2 * x), (4, 3, 1 * x)]
    N = 5
    graph = Graph(N, edges, x)
    source = 0
    dest = 2
    BFS(graph, source, dest)
 


[[5, 4], [2, 6, 4], [], [], [9, 3], [10], [11], [], [], [2], [1], [3], [], [], []]
least cost path between 0  and  2 is  0 4 2 having cost 3


In [38]:
# Find maximum cost path in graph from given source to destination
""" Given a weighted graph, find the maximum cost path from given source to the destination that is grater than a given 
   integer x. The path should not contain any cycles."""
from collections import deque, defaultdict
class Graph:
    def __init__(self):
        self.graph = defaultdict(list)
        self.adj = []
    def print_graph(self):
        return self.graph
    def addEdge(self, v, u, weight):
        self.graph[v].append((u, weight))
        self.graph[u].append((v, weight))
        
    def count_vertices(self):
        count = 0
        visit = set()
        for i in self.graph:
            if i not in visit:
                visit.add(i)
                count += 1
            for j in self.graph[i]:
                if j[0] not in visit:
                    visit.add(j[0])
                    count += 1
        return count
    
    def BFS(self, source, k):
        q = deque()
        vertices = set([0])
        q.append((source, 0, vertices))
        max_cost = float("-inf")
        while q:
            v, cost, vertices = q.popleft()
            if cost > max_cost: max_cost = cost
            
            for edge in self.graph[v]:
                if not edge[0] in vertices:
                    s = set(vertices)
                    s.add(edge[0])
                    q.append((edge[0], cost + edge[1], s))
        return max_cost if max_cost != float('-inf') else "all paths of cost less than given cost"
                
g = Graph()
g.addEdge(0,6,8)
g.addEdge(0,1,5)
g.addEdge(1,6,3)
g.addEdge(1,5,5)
g.addEdge(1,2,7)
g.addEdge(2,3,-8)
g.addEdge(3,4,10)
g.addEdge(5,2,-1)
g.addEdge(5,3,9)
g.addEdge(5,4,1)
g.addEdge(6,5,2)
g.addEdge(7,6,9)
g.addEdge(7,1,6)
print(g.print_graph()) 
print(g.count_vertices())
print(g.BFS(0, 25))

defaultdict(<class 'list'>, {0: [(6, 8), (1, 5)], 6: [(0, 8), (1, 3), (5, 2), (7, 9)], 1: [(0, 5), (6, 3), (5, 5), (2, 7), (7, 6)], 5: [(1, 5), (2, -1), (3, 9), (4, 1), (6, 2)], 2: [(1, 7), (3, -8), (5, -1)], 3: [(2, -8), (4, 10), (5, 9)], 4: [(3, 10), (5, 1)], 7: [(6, 9), (1, 6)]})
8
48


In [47]:
# determine negative weight cycle in the graph:
from collections import defaultdict
class Graph:
    def __init__(self):
        self.graph = defaultdict(list)
    def addEdge(self, u, v, weight):
        self.graph[u].append((v, weight))
    def print_graph(self):
        return self.graph
    def count_vertices(self):
        count = 0
        visit = set()
        for i in self.graph:
            if i not in visit:
                visit.add(i)
                count += 1
            for j in self.graph[i]:
                if j[0] not in visit:
                    visit.add(j[0])
                    count += 1
        return count
    
    def findNegativeCycle(self):
        N = self.count_vertices()
        cost = [float('inf')]*N
        cost[source] = 0
        # do the relaxation for N-1 times
        for _ in range(N-1):
            for v in self.graph:
                for u in self.graph[v]:
                    u1, w = u
                    if cost[v] != float("inf") and cost[u1] > cost[v]+w:
                        cost[u1] = cost[v] + w
        # do the relaxation for one more time
        for v in self.graph:
            for u in self.graph[v]:
                if cost[v] != float("inf") and cost[u1] > cost[v]+w:
                    return True
        return False
        
g = Graph()
g.addEdge(1,0,4)
g.addEdge(1,2,-3)
g.addEdge(2,3,2)
g.addEdge(3,1,-1)
g.addEdge(0,2,-2)
print(g.print_graph())
print(g.count_vertices())
print(g.findNegativeCycle())

defaultdict(<class 'list'>, {1: [(0, 4), (2, -3)], 2: [(3, 2)], 3: [(1, -1)], 0: [(2, -2)]})
4
True


In [54]:
# Least cost path in a given digraph from given source to destination having exactly m edges:

from collections import deque
class Graph:
    def __init__(self):
        self.graph = defaultdict(list)
    def addEdge(self, u, v, w):
        self.graph[u].append((v, w))
    def print_graph(self):
        return self.graph
    def count_vertices(self):
        visit = set()
        count = 0
        for i in self.graph:
            if i not in visit:
                visit.add(i)
                count+=1
            for j in self.graph[i]:
                if j[0] not in visit:
                    visit.add(j[0])
                    count+=1
        return count
    
    def BFS(self,source,dest, m):
        q = deque()
        q.append((source,0, 0)) # push vertex, depth, cost
        min_cost = float("inf")
        while q:
            vertex, depth, cost = q.popleft()
            if vertex == dest and depth == m:
                min_cost = min(cost, min_cost)
                
            if depth > m: break
                
            for u in self.graph[vertex]:
                u1, w = u
                q.append((u1,depth+1,cost+w))
        return min_cost
    
g = Graph()
g.addEdge(0, 6, -1)
g.addEdge(0, 1, 5)
g.addEdge(1,6,3)
g.addEdge(1,5,5)
g.addEdge(1,2,7)
g.addEdge(2,3,8)
g.addEdge(3,4,10)
g.addEdge(5,2, -1)
g.addEdge(5,3,9)
g.addEdge(5,4,1)
g.addEdge(6,5,2)
g.addEdge(7, 6, 9)
g.addEdge(7,1,6)
print(g.count_vertices())
print(g.print_graph())
print(g.BFS(0, 3, 4))

8
defaultdict(<class 'list'>, {0: [(6, -1), (1, 5)], 1: [(6, 3), (5, 5), (2, 7)], 2: [(3, 8)], 3: [(4, 10)], 5: [(2, -1), (3, 9), (4, 1)], 6: [(5, 2)], 7: [(6, 9), (1, 6)]})
8


In [58]:
#Find the path between given vertices in a directed graph

from collections import deque, defaultdict
class Graph:
    def __init__(self):
        self.graph = defaultdict(list)
    def addEdge(self, u, v):
        self.graph[u].append(v)
    def print_graph(self):
        return self.graph
    def count_vertices(self):
        count = 0
        visit = set()
        for i in self.graph:
            if i not in visit:
                visit.add(i)
                count += 1
            for j in self.graph[i]:
                if j not in visit:
                    visit.add(j)
                    count += 1
        return count
    
    def path_between_nodes(self, source, dst): # apply BFS
        q = deque()
        q.append(source)
        visited = set()
        while q:
            current = q.popleft()
            if current == dst: return True
            for vertex in self.graph[current]:
                if vertex not in visited:
                    q.append(vertex)
                    visited.add(vertex)
        return False
                
g = Graph()
g.addEdge(0,3)
g.addEdge(1,0)
g.addEdge(1,2)
g.addEdge(1,4)
g.addEdge(2,7)
g.addEdge(3,4)
g.addEdge(3,5)
g.addEdge(4,3)
g.addEdge(4,6)
g.addEdge(5,6)
g.addEdge(6,7)
print(g.count_vertices())
print(g.print_graph())
print(g.path_between_nodes(1,5))

8
defaultdict(<class 'list'>, {0: [3], 1: [0, 2, 4], 2: [7], 3: [4, 5], 4: [3, 6], 5: [6], 6: [7]})
True


In [18]:
# Find all Possible Topological Orderings of a DAG:
from collections import defaultdict
class Graph:
    def __init__(self):
        self.graph = defaultdict(list)
    
    def addEdge(self, u, v):
        self.graph[u].append(v)
    
    def print_graph(self):
        return self.graph
    
    def count_vertices(self):
        visit, count = set(), 0
        for i in self.graph:
            if i not in visit:
                visit.add(i)
                count += 1
            for j in self.graph[i]:
                if j not in visit:
                    visit.add(j)
                    count += 1
        return count
    def findAllTopologicalOrders(self, path, discovered, n, indegree):
        for v in range(n):
            if indegree[v] == 0 and not discovered[v]:
                for u in self.graph[v]:
                    indegree[u] -= 1
                path.append(v)
                discovered[v] = True
                
                self.findAllTopologicalOrders(path, discovered, n, indegree)
                
                for u in self.graph[v]:
                    indegree[u]+=1
                    
                path.pop()
                discovered[v] = False
                
        if len(path) == n: print(path)
                
    
    """  [0, 0, 0, 0, 0, 0, 0, 0]
         [T, T, T, T, T, T, T, T]
         {0: [6], 1: [2, 4, 6], 3: [0, 4], 5: [1], 7: [0, 1]}
         path = [3, 5, 7, 0, 1, 2, 4, 6]
         
         f([], [F, F, F, F, F, F, F, F], 8, [2, 2, 1, 0, 2, 0, 2, 0])
         f([3], [F, F, F, T, F, F, F, F], 8, [1, 2, 1, 0, 1, 0, 2, 0])
         f([3,5], [F, F, F, T, F, T, F, F],  [1, 1, 1, 0, 1, 0, 2, 0])
         f([3,5,7], [F, F, F, T, F, T, F, T], [0, 0, 1, 0, 1, 0, 2, 0])
         f([3, 5, 7, 0],[T, F, F, T, F, T, F, T], [0, 0, 1, 0, 1, 0, 1, 0])
         f([3, 5, 7, 0, 1], [T, T, F, T, F, T, F, T], [0, 0, 0, 0, 0, 0, 0, 0])
         f([3, 5, 7, 0, 1, 2], [T, T, T, T, F, T, F, T],[0, 0, 0, 0, 0, 0, 0, 0])
         f([3, 5, 7, 0, 1, 2, 4], [T, T, T, T, T, T, F, T],[0, 0, 0, 0, 0, 0, 0, 0]))
         f([3, 5, 7, 0, 1, 2, 4, 6],[T, T, T, T, T, T, T, T], [0, 0, 0, 0, 0, 0, 0, 0])
         """
    def printAllTopologicalOrders(self):
        N = self.count_vertices()
        discovered = [False]*N
        path = []
        indegree = [0]*N
        for i in self.graph:
            for j in self.graph[i]:
                indegree[j] += 1
        print(indegree)      
        self.findAllTopologicalOrders(path, discovered, N, indegree)
    
g = Graph()
g.addEdge(0,6)
g.addEdge(1,2)
g.addEdge(1,4)
g.addEdge(1,6)
g.addEdge(3,0)
g.addEdge(3,4)
g.addEdge(5,1)
g.addEdge(7,0)
g.addEdge(7,1)
print(g.print_graph())
print(g.count_vertices())
print(g.printAllTopologicalOrders())

defaultdict(<class 'list'>, {0: [6], 1: [2, 4, 6], 3: [0, 4], 5: [1], 7: [0, 1]})
8
[2, 2, 1, 0, 2, 0, 2, 0]
[3, 5, 7, 0, 1, 2, 4, 6]
[3, 5, 7, 0, 1, 2, 6, 4]
[3, 5, 7, 0, 1, 4, 2, 6]
[3, 5, 7, 0, 1, 4, 6, 2]
[3, 5, 7, 0, 1, 6, 2, 4]
[3, 5, 7, 0, 1, 6, 4, 2]
[3, 5, 7, 1, 0, 2, 4, 6]
[3, 5, 7, 1, 0, 2, 6, 4]
[3, 5, 7, 1, 0, 4, 2, 6]
[3, 5, 7, 1, 0, 4, 6, 2]
[3, 5, 7, 1, 0, 6, 2, 4]
[3, 5, 7, 1, 0, 6, 4, 2]
[3, 5, 7, 1, 2, 0, 4, 6]
[3, 5, 7, 1, 2, 0, 6, 4]
[3, 5, 7, 1, 2, 4, 0, 6]
[3, 5, 7, 1, 4, 0, 2, 6]
[3, 5, 7, 1, 4, 0, 6, 2]
[3, 5, 7, 1, 4, 2, 0, 6]
[3, 7, 0, 5, 1, 2, 4, 6]
[3, 7, 0, 5, 1, 2, 6, 4]
[3, 7, 0, 5, 1, 4, 2, 6]
[3, 7, 0, 5, 1, 4, 6, 2]
[3, 7, 0, 5, 1, 6, 2, 4]
[3, 7, 0, 5, 1, 6, 4, 2]
[3, 7, 5, 0, 1, 2, 4, 6]
[3, 7, 5, 0, 1, 2, 6, 4]
[3, 7, 5, 0, 1, 4, 2, 6]
[3, 7, 5, 0, 1, 4, 6, 2]
[3, 7, 5, 0, 1, 6, 2, 4]
[3, 7, 5, 0, 1, 6, 4, 2]
[3, 7, 5, 1, 0, 2, 4, 6]
[3, 7, 5, 1, 0, 2, 6, 4]
[3, 7, 5, 1, 0, 4, 2, 6]
[3, 7, 5, 1, 0, 4, 6, 2]
[3, 7, 5, 1, 0, 6, 2, 4]
[3, 7, 5, 1, 0, 

In [28]:
# Find correct order of alphabets in a given dictionary of ancient origin ************************

class Graph:
    def __init__(self, N):
        self.adj = [[] for _ in range(N)]

def findDictionaryAlphabetsOrder(dictionary):
    d = {}
    k = 0
    for word in dictionary:
        for s in word:
            d.setdefault(s,k)
            k+=1
            
    graph = Graph(N)
    for i in range(1, len(dictionary)):
        prev = dictionary[i-1]
        current = dictionary[i]
        j = 0
        while j < len(prev) and j < len(current):
            if prev[j] != current[j]:
                graph.adj[d[prev[j]]].append(d[current[j]])
                break
            j+=1
            
    reverse = dict((v, k) for k,v in d.items())
    print_graph(graph, reverse)
    doTopologicalSorting(graph, reverse)
    
def doTopologicalSorting(graph, dict1):
    departure = [-1]*2*N
    discovered = [False]*N
    time = 0
    for i in range(N):
        if not discovered[i] and len(graph.adj[i]):
            time = DFS(graph, i, discovered, departure, time)
            
    for i in reversed(range(2*N)):
        if departure[i] != -1:
            print(dict1[departure[i]], end = " ")
            
def DFS(graph, v, discovered, departure, time):
    discovered[v] = True
    time += 1
    for u in graph.adj[v]:
        if not discovered[u]:
            time = DFS(graph, u, discovered, departure, time)
            
    departure[time] = v
    return time+1

def print_graph(graph, dict1):
    for i in range(N):
        if graph.adj[i]:
            print(dict1[i], "->", [dict1[v] for v in graph.adj[i]])
            
            
if __name__ == '__main__':
    N = 100
    dictionary = [["¥", "€", "±"],
        ["€", "±", "€"],
        ["€", "±", "‰", "ð"],
        ["ð", "ß"],
        ["±", "±", "ð"],
        ["±", "ß", "ß"]
    ]
    
    findDictionaryAlphabetsOrder(dictionary)
    

¥ -> ['€']
€ -> ['‰', 'ð']
± -> ['ß']
ð -> ['±']
¥ € ð ± ß ‰ 

In [34]:
# Find longest path in a Directed Acyclic Graph (DAG)

from collections import defaultdict
class Graph:
    def __init__(self):
        self.graph = defaultdict(list)
        
    def addEdge(self, u, v, w):
        self.graph[u].append((v, w))
        
    def print_graph(self):
        return self.graph
    
    def count_vertices(self):
        visit, count = set(), 0
        for i in self.graph:
            if i not in visit:
                visit.add(i)
                count+=1
            for j in self.graph[i]:
                if j[0] not in visit:
                    visit.add(j[0])
                    count+=1
        return count
    
    def DFS(self, v, discovered, departure, time):
        discovered[v] = True
        for e in self.graph[v]:
            u, w = e
            if not discovered[u]:
                time = self.DFS(u, discovered, departure, time)
                
        departure[time] = v
        time += 1
        return time
    
    def find_longest_path(self, source):
        N = self.count_vertices()
        departure = [-1]*N
        discovered = [False]*N
        time = 0
        for i in range(N):
            if not discovered[i]:
                time = self.DFS(i, discovered, departure, time)
                
        cost = [float('inf')]*N
        cost[source] = 0
        
        for i in reversed(range(N)):
            v = departure[i]
            for e in self.graph[v]:
                u, w = e
                w *= -1
                if cost[v] != float('inf') and cost[v]+w < cost[u]:
                    cost[u] = cost[v]+w
        
        for i in range(N):
            print("dist", (source, i), "=", (cost[i]*-1))
    
g = Graph()
g.addEdge(0,6,2)
g.addEdge(1,2,-4)
g.addEdge(1,4,1)
g.addEdge(1,6,8)
g.addEdge(3,0,3)
g.addEdge(3,4,5)
g.addEdge(5,1,2)
g.addEdge(7,0,6)
g.addEdge(7,1,-1)
g.addEdge(7,3,4)
g.addEdge(7,5,-4)
print(g.print_graph())
print(g.count_vertices())
print(g.find_longest_path(7))

defaultdict(<class 'list'>, {0: [(6, 2)], 1: [(2, -4), (4, 1), (6, 8)], 3: [(0, 3), (4, 5)], 5: [(1, 2)], 7: [(0, 6), (1, -1), (3, 4), (5, -4)]})
8
dist (7, 0) = 7
dist (7, 1) = -1
dist (7, 2) = -5
dist (7, 3) = 4
dist (7, 4) = 9
dist (7, 5) = -4
dist (7, 6) = 9
dist (7, 7) = 0
None


In [1]:
# Construct a directed graph from undirected graph that satisfies given constraints

from collections import deque, defaultdict
class Graph:
    def __init__(self):
        self.graph = defaultdict(list)
    
    def addEdge(self, u, v):
        self.graph[u].append(v)
        self.graph[v].append(u)
        
    def print_graph(self):
        return self.graph
    
    def count_vertices(self):
        visit, count = set(), 0
        for i in self.graph:
            if i not in visit:
                visit.add(i)
                count += 1
            for j in self.graph[i]:
                if j not in visit:
                    visit.add(j)
                    count += 1
        return count
    
    def BFS(self,v):
        N = self.count_vertices()
        discovered = [False]*(N+1)
        discovered[v] = True
        edges = []
        q = deque()
        q.append(v)
        while q:
            v = q.popleft()
            for u in self.graph[v]:
                if not discovered[u]:
                    discovered[u] = True
                    q.append(u)
                    edges.append((v, u))
        return edges
        
    
g = Graph()
g.addEdge(1,2)
g.addEdge(1,3)
g.addEdge(2,4)
g.addEdge(3,4)
g.addEdge(3,5)
g.addEdge(4,5)
g.addEdge(4,6)
print(g.print_graph())
print(g.count_vertices())
print(g.BFS(1))

defaultdict(<class 'list'>, {1: [2, 3], 2: [1, 4], 3: [1, 4, 5], 4: [2, 3, 5, 6], 5: [3, 4], 6: [4]})
6
[(1, 2), (1, 3), (2, 4), (3, 5), (4, 6)]


In [4]:
# Print all k-colorable configurations of the graph (Vertex coloring of graph)

class Graph:
    def __init__(self,edges, N):
        self.adjList = [[] for _ in range(N)]
        for (src, dst) in edges:
            self.adjList[src].append(dst)
            self.adjList[dst].append(src)
            
def isSafe(graph, color, v, c):
    for u in graph.adjList[v]:
        if color[u] == c:
            return False
    return True
    
def k_colorable(g, color, k, v, N):
    if v == N:
        print([COLORS[color[v]] for v in range(N)])
        return
    for c in range(1, k+1):
        if isSafe(g, color, v, c):
            color[v] = c
            k_colorable(g, color, k, v+1, N)
            color[v] = 0
            
            
    """ kcolor(g, [none, none, none, none, none, none], 3, v = 0, N)
        for 1,2,3
           safe(g, [none, none, none, none, none, none], v = 0, 1)-> True , [1, none, none, none, none, none]
           kcolor(g,[1, none, none, none, none, none],3,1,N)
           for 1, 2, 3
              safe(g,[1, none, none, none, none, none],1,1) -> False
              safe(g,[1, none, none, none, none, none],1,2) -> True, [1, 2, none, none, none, none]
              kcolor(g,[1, 2, none, none, none, none], k, 2, N)
              for 1,2,3
                 safe(g, [1, 2, none, none, none, none],2,1) -> True, [1, 2, 1, none, none, none]
                 kcolor(g, [1, 2, 1, none, none, none],k, 3,N)
                 for 1,2,3:
                    safe(g,[1, 2, 1, none, none, none],3,1) -> False
                    safe(g,[1, 2, 1, none, none, none],3,2) -> False
                    safe(g,[1, 2, 1, none, none, none],3,3) -> True, [1, 2, 1, 3, none, none]
                    kcolor(g,[1, 2, 1, 3, none, none],k,4,N)
                    for 1, 2, 3:
                       safe(g,[1, 2, 1, 3, none, none],4,1)->False
                       safe(g,[1, 2, 1, 3, none, none],4,2)->False
                       safe(g,[1, 2, 1, 3, none, none],4,3)->True,[1, 2, 1, 3, 3, none]
                       kcolor(g,[1, 2, 1, 3, 3, none], k,5,N)
                       for 1,2,3:
                          safe(g,[1, 2, 1, 3, 3, none],5,1)->False
                          safe(g,[[1, 2, 1, 3, 3, none],5,2])-> T,[1, 2, 1, 3, 3, 2]
                          kcolor(g,[1, 2, 1, 3, 3, 2],k,6(v+1),N): v==N: print_colors [1, 2, 1, 3, 3, 0]
                          safe(g,[1, 2, 1, 3, 3, 0],5,3)-> False
    
    
    """
if __name__ == '__main__':
    edges = [(0, 1), (0, 4),(0, 5), (4, 5),(1, 4), (1, 3),(2, 3), (2, 4)]
    COLORS = ["", "BLUE", "GREEN", "RED", "YELLOW", "ORANGE", "PINK",
              "BLACK", "BROWN", "WHITE", "PURPLE"]
    N = 6
    g = Graph(edges, N)
    k = 3
    color = [None]*N
    k_colorable(g, color, k, 0, N)

['BLUE', 'GREEN', 'BLUE', 'RED', 'RED', 'GREEN']
['BLUE', 'GREEN', 'GREEN', 'BLUE', 'RED', 'GREEN']
['BLUE', 'GREEN', 'GREEN', 'RED', 'RED', 'GREEN']
['BLUE', 'RED', 'BLUE', 'GREEN', 'GREEN', 'RED']
['BLUE', 'RED', 'RED', 'BLUE', 'GREEN', 'RED']
['BLUE', 'RED', 'RED', 'GREEN', 'GREEN', 'RED']
['GREEN', 'BLUE', 'BLUE', 'GREEN', 'RED', 'BLUE']
['GREEN', 'BLUE', 'BLUE', 'RED', 'RED', 'BLUE']
['GREEN', 'BLUE', 'GREEN', 'RED', 'RED', 'BLUE']
['GREEN', 'RED', 'GREEN', 'BLUE', 'BLUE', 'RED']
['GREEN', 'RED', 'RED', 'BLUE', 'BLUE', 'RED']
['GREEN', 'RED', 'RED', 'GREEN', 'BLUE', 'RED']
['RED', 'BLUE', 'BLUE', 'GREEN', 'GREEN', 'BLUE']
['RED', 'BLUE', 'BLUE', 'RED', 'GREEN', 'BLUE']
['RED', 'BLUE', 'RED', 'GREEN', 'GREEN', 'BLUE']
['RED', 'GREEN', 'GREEN', 'BLUE', 'BLUE', 'GREEN']
['RED', 'GREEN', 'GREEN', 'RED', 'BLUE', 'GREEN']
['RED', 'GREEN', 'RED', 'BLUE', 'BLUE', 'GREEN']


In [12]:
#Print all Hamiltonian path present in a graph

from collections import deque, defaultdict
class Graph:
    def __init__(self):
        self.graph = defaultdict(list)
    
    def addEdge(self, u, v):
        self.graph[u].append(v)
        self.graph[v].append(u)
        
    def print_graph(self):
        return self.graph
    
    def count_vertices(self):
        visit, count = set(), 0
        for i in self.graph:
            if i not in visit:
                visit.add(i)
                count += 1
            for j in self.graph[i]:
                if j not in visit:
                    visit.add(j)
                    count += 1
        return count
    
    def print_hamilton_paths(self, source):
        self.N = self.count_vertices()
        self.visit = [False]*self.N
        self.visit[source] = True
        self.path = [source]
        def function(source):
            if len(self.path) == self.N:
                print(self.path)
                return 
            for w in self.graph[source]:
                if not self.visit[w]:
                    self.visit[w] = True
                    self.path.append(w)
                    function(w)
                    self.visit[w] = False
                    self.path.pop()
        function(source)
                
    
g = Graph()
g.addEdge(0,1)
g.addEdge(0,2)
g.addEdge(0,3)
g.addEdge(1,2)
g.addEdge(1,3)
g.addEdge(2,3)
print(g.print_graph())
print(g.count_vertices())
print(g.print_hamilton_paths(0))

defaultdict(<class 'list'>, {0: [1, 2, 3], 1: [0, 2, 3], 2: [0, 1, 3], 3: [0, 1, 2]})
4
[0, 1, 2, 3]
[0, 1, 3, 2]
[0, 2, 1, 3]
[0, 2, 3, 1]
[0, 3, 1, 2]
[0, 3, 2, 1]
None


In [10]:
# Graph coloring problem
""" The graph coloring is also called the vertex coloring is the way of coloring the vertices of graph such that no adjacent
vertices share same color."""

class Graph:
    def __init__(self,edges, N):
        self.adj = [[] for _ in range(N)]
        for (src,dst) in edges:
            self.adj[src].append(dst)
            self.adj[dst].append(src)
    
def color_Graph(graph):
    print(graph.adj)
    result = {}
    for u in range(N):
        assigned = set([result.get(i) for i in graph.adj[u] if i in result])
        color = 1
        for c in assigned:
            if color != c:
                break
            color += 1
        result[u] = color
       
    for v in range(N):
        print("color assigned to vertex", v, "is", colors[result[v]])
        
   
    
if __name__ == '__main__':
    
    colors = ["", "BLUE", "GREEN", "RED", "YELLOW", "ORANGE", "PINK",
              "BLACK", "BROWN", "WHITE", "PURPLE", "VOILET"]
    edges = [(0, 1), (0, 4), (0, 5), (4, 5), (1, 4), (1, 3), (2, 3), (2, 4)]
    N = 6
    graph = Graph(edges, N)
    color_Graph(graph)
 

[[1, 4, 5], [0, 4, 3], [3, 4], [1, 2], [0, 5, 1, 2], [0, 4]]
color assigned to vertex 0 is BLUE
color assigned to vertex 1 is GREEN
color assigned to vertex 2 is BLUE
color assigned to vertex 3 is RED
color assigned to vertex 4 is RED
color assigned to vertex 5 is GREEN


In [2]:
1//2

0